# Origence Client Model 1

With LTV feature

Total rows: 587636\
Train: 356330 rows (60.64%) start date: 2020-01-01 end date: 2022-03-13\
Valid: 115341 rows (19.63%) start date: 2022-03-13 end date: 2022-07-28\
Test:  115965 rows (19.73%) start date: 2022-07-28 end date: 2023-03-31


In [1]:
import os
import pandas as pd
import json

from model_engine.assets.utils import load_asset
from model_engine.io.loaders import load_json
from model_engine.power.post_sale import PowerModelBuilder
from model_engine.analysis.suggesters import evaluate_datesplits
import warnings

from zaml.common.utils import load_state

import model_engine, zaml
model_engine.__version__, zaml.__version__

/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/zamlexplain/utils_compile.py:23: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/lyt/.conda/envs/penfed_modeling/lib/python3.1

('v1.13.0', '34.5.2')

In [13]:
from model_engine.model_builder.asset_parser import asset_parser
from model_engine.model_builder import ModelBuilder

In [14]:
with open('project_info.json', "r") as f:
    project_info = json.load(f)
input_client_name = project_info['client_name']
model_type = project_info['national_model_category'] # options are 'standard_model', 'mega_model', 'standard_model_member_data', 'standard_model_ln_data', 'standard_model_ln_member_data'- None defaults to 'standard_model'

print(input_client_name, model_type)

origencepocv2 standard_model_member_data


In [21]:
temp = pd.read_csv('/d/shared/silver_projects/firsthawaiianbank/creditcard/shared_data/processed/target/final_target.csv')
temp.final_DQ60_m24.value_counts()

final_DQ60_m24
0.0    54650
1.0     3522
Name: count, dtype: int64

In [22]:
temp.groupby('flgFunded').final_DQ60_m24.value_counts(normalize = True)

flgFunded  final_DQ60_m24
0          0.0               0.876600
           1.0               0.123400
1          0.0               0.954267
           1.0               0.045733
Name: proportion, dtype: float64

In [25]:
temp[temp.final_DQ60_m24.notna()].appDate.min(), temp[temp.final_DQ60_m24.notna()].appDate.max()

('2016-01-04', '2021-06-18')

In [20]:
target = pd.read_parquet('/d/shared/silver_projects/firsthawaiianbank/perfvldSMB/shared_data/processed/target/final_target.parquet')
target.groupby('flgFunded').final_DQ60_m24.value_counts(normalize = True)

flgFunded  final_DQ60_m24
0          0.0               0.878378
           1.0               0.121622
1          0.0               0.989474
           1.0               0.010526
Name: proportion, dtype: float64

## 1. Unfold Model

In [6]:
input_model_id = 'clientmodel1a'
input_target_name = 'final_DQ60_m24'


final_model_output_path = f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/{input_model_id}'
aux_model_output_path = f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/auxiliary_model_artifacts/autoloan/{input_model_id}'


print(f'Final model path: {final_model_output_path}')
print(f'aux model path: {aux_model_output_path}')

Final model path: /d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloanclientmodel1a
aux model path: /d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/auxiliary_model_artifacts/autoloan/clientmodel1a


In [8]:
asset = json.load(open(f'./{input_model_id}_asset.json'))

In [9]:
model_builder = ModelBuilder(asset)
model_builder.configure_run(final_model_output_path, 
                            cache=True) 

state = asset_parser(asset)

input_asset = (json.dumps(state['input_asset'], indent=4))
print(input_asset)

{
    "data": {
        "trade": {
            "asset": "equifax/cms_6/trade.json",
            "data": "#/home/lyt/CLIENTS/client-project-penfed/modeling/client_models/sample_files/trade_sampled_files.json",
            "io_params": {
                "drop_duplicates": true,
                "keep_features": [
                    "trade_acctType__count",
                    "trade_blncAmt__mean_by_acctType_auto",
                    "trade_blncAmt__min_by_acctType_education",
                    "trade_blncAmt_accts_never_dq__max",
                    "trade_blncAmt_accts_never_dq__max_by_ecoa_joint",
                    "trade_blncAmt_accts_never_dq__max_by_prtfType_mortgage",
                    "trade_blncAmt_accts_never_dq__mean_by_acctType_unsecure",
                    "trade_blncAmt_accts_never_dq__mean_by_ecoa_joint",
                    "trade_blncAmt_accts_never_dq__sum",
                    "trade_blncAmt_accts_never_dq__sum_by_prtfType_mortgage",
                    "trade_

In [11]:
from model_engine.model_builder import build_model

output = build_model.build_model(asset, final_model_output_path)

parsing model-builder asset


Configuring model builder


INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <valid_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <valid_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engin

building model


INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:00.755584
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.000877
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000561
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list>...
INFO:zaml.artifact_engine.logger:Finished <parsed_exclusion_list>, total time spent: 0:00:00.000630
INFO:zaml.artifact_engine.logger:Executing BivariateFeParser <parsed_biv_fe_instructions>...
INFO:zaml.artifact_engine.logger:Finished <parsed_biv_fe_instructions>, total time spent: 0:00:00.000587
INFO:zaml.artifact_engine.logger:Executing NeutralizationListParser <parsed_neutralization_list>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifa

...

-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 271
Time spent: 10.800s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 271
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <valid_fe_data>, total time spent: 0:00:48.296201
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.000s
-------------------------
Name: client_data
Transformer type: None
Number of features: 431
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 216
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.099s
-------------------------
Name: bnkr FE
Transformer type: ManyToOneEngine
Number of features: 8
Time spent: 0.018s
-------------------------
Name: client_data FE
T

-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 271
Time spent: 11.205s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 271
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:01:16.716324
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...
INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:11.081964
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.003913
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000401
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:05.702183
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.000719
INFO:zaml.artifa

In [12]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
#     for i in ['train', 'valid', 'test']:
    for i in ['train','test',]:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(final_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.8200320583949494}
test_auc {'auc': 0.7889459076566214}
train_ks {'ks': 0.4860048525836357}
test_ks {'ks': 0.43388780516871034}


In [26]:
final_model_output_path

'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloanclientmodel1a'

## 2. Fold valid Model

In [15]:
fold_valid_model_id = 'clientmodel1b'
fold_valid_model_output_path = f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/{fold_valid_model_id}'
fold_valid_model_output_path

'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/clientmodel1b'

In [16]:
base_model_id = 'clientmodel1a'
asset = json.load(open(f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/{base_model_id}/asset.json'))
keep_features = json.load(open(f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/{base_model_id}/keep_features.json'))
best_ntree = json.load(open(f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/{base_model_id}/best_model_params.json'))
best_ntree['best_ntree_limit']

5352

In [17]:
asset['config']['data_split'] 

{'train': {'start_date': '2020-01-01', 'end_date': '2022-03-13'},
 'valid': {'start_date': '2022-03-13', 'end_date': '2022-07-28'},
 'test': {'start_date': '2022-07-28', 'end_date': '2023-03-31'}}

In [18]:
asset['config']['data_split'] = {
    'train': {'start_date': '2020-01-01', 'end_date': '2022-07-28'},
    'test': {'start_date': '2022-07-28', 'end_date': '2023-03-31'}}

In [19]:
asset['config']['pipeline_factory'] =  {
      "transformers": [
        {
          "zaml_class": "LevelSelection",
          "params": {
            "thresh": 0.01,
            "change_to": "Other",
            "encoding": "onehot"
          }
        },
        {
          "zaml_class": "FillNA",
          "params": {
            "replace_by": -1,
            "add_flags": False
          }
        },
        {
          "zaml_class": "GeneralFeatureSelection",
          "params": {
            "features": [
              "app_is_client"
            ]
          }
        }
      ],
      "model": {
        "zaml_class": "XGBoostModel",
        "params": {
          "n_estimators": best_ntree['best_ntree_limit'],
          "learning_rate": 0.01,
          "max_depth": 3,
          "subsample": 0.5,
          "scale_pos_weight": 2.5,
          "colsample_bytree": 0.05,
          "min_child_weight": 350,
          "seed": 12,
          "eval_metric": "auc"
        }
      }
    }

In [20]:
asset['config']['pipeline_factory']['model']['params']['n_estimators'] 

5352

In [21]:
asset['config'].pop('mandatory_features')
asset['config']['fold_valid'] = True
asset['config']['enforce_artifact_validation'] = True
asset['config']['base_features'] = f'/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/{base_model_id}/keep_features.json'


In [22]:
for t, l in keep_features.items():
    print(t)
    if t != 'client_data': 
        asset['data'][t]['io_params']['keep_features'] = l
    else: 
        continue

trade
inq
collec
bnkr
member
client_data


In [23]:
asset['data']['app']

{'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
  'table_name': 'app',
  'table_type': 'one_to_one',
  'feature_engineering': 'one_to_one'},
 'data': '#/home/lyt/CLIENTS/client-project-penfed/modeling/client_models/sample_files/app_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_features': ['appDate',
   'is_client',
   'bisgWhite',
   'bisgBlack',
   'bisgAian',
   'bisgApi',
   'bisgHispanic',
   'bisgMulti',
   'prob_male',
   'prob_female',
   'prob_<62',
   'prob_62+',
   'VS30',
   'VS40',
   'perf_state_cde',
   'perf_acct_typ_cde',
   'kob',
   'flg_exist_fe_trade',
   'ZEST_KEY'],
  'memory_efficient': True}}

In [24]:
asset['data']['client_data']

{'data': '#/home/lyt/CLIENTS/client-project-penfed/modeling/client_models/sample_files/client_data_sampled_files.json',
 'asset': {'feature_engineering': 'one_to_one',
  'validator': 'base',
  'table_type': 'app',
  'info': {'key': 'ZEST_KEY', 'numerics': ['LTV_NORM']}}}

In [25]:
model_builder = ModelBuilder(asset)
model_builder.configure_run(fold_valid_model_output_path, 
                            cache=True) 

state = asset_parser(asset)

input_asset = (json.dumps(state['input_asset'], indent=4))
print(input_asset)

{
    "data": {
        "trade": {
            "asset": "equifax/cms_6/trade.json",
            "data": "#/home/lyt/CLIENTS/client-project-penfed/modeling/client_models/sample_files/trade_sampled_files.json",
            "io_params": {
                "drop_duplicates": true,
                "keep_features": [
                    "trade_acctType__count",
                    "trade_blncAmt__mean_by_acctType_auto",
                    "trade_blncAmt__min_by_acctType_education",
                    "trade_blncAmt_accts_never_dq__max",
                    "trade_blncAmt_accts_never_dq__max_by_ecoa_joint",
                    "trade_blncAmt_accts_never_dq__max_by_prtfType_mortgage",
                    "trade_blncAmt_accts_never_dq__mean_by_acctType_unsecure",
                    "trade_blncAmt_accts_never_dq__mean_by_ecoa_joint",
                    "trade_blncAmt_accts_never_dq__sum",
                    "trade_blncAmt_accts_never_dq__sum_by_prtfType_mortgage",
                    "trade_

In [26]:
with open (os.path.join(fold_valid_model_output_path, 'input_configuration.json'), 'w') as outfile:
    outfile.write(input_asset)

In [27]:
from model_engine.model_builder import build_model

output = build_model.build_model(asset, fold_valid_model_output_path)

parsing model-builder asset


Configuring model builder


INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <n_tiers>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <n_tiers>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing Input

building model


INFO:zaml.artifact_engine.logger:Finished <versions>, total time spent: 0:00:05.333423
INFO:zaml.artifact_engine.logger:Executing MonotonicConstraintsListParser <parsed_monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:01.077993
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.001147
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000379
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list>...
INFO:zaml.artifact_engine.logger:Finished <parsed_exclusion_list>, total time spent: 0:00:00.000463
INFO:zaml.artifact_engine.logger:Executing BivariateFeParser <parsed_biv_fe_instructions>...
INFO:zaml.artifact_engine.logger:Finished <parsed_biv_fe_instructions>, total time spent: 0:00:

...

-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.000s
-------------------------
Name: client_data
Transformer type: None
Number of features: 431
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 216
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.035s
-------------------------
Name: bnkr FE
Transformer type: ManyToOneEngine
Number of features: 8
Time spent: 0.019s
-------------------------
Name: client_data FE
T


INFO:zaml.artifact_engine.logger:Finished <train_fe_data>, total time spent: 0:00:43.906885
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 271
Time spent: 3.288s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 271
Time spent: 0.001s
-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: bnkr
Transformer type: None
Number of features: 8
Time spent: 0.000s
-------------------------
Name: client_data
Transformer type: None
Number of features: 431
Time spent: 0.000s
-------------------------
Name: collec
Transformer type: None
Number of features: 5
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 13
Time spent: 0.000s
-------------------------
Name: member
Transformer type: None
Number of features: 28
Time spent: 0.000s
-------------------------
Name: trade
Transformer type: None
Number of features: 216
Time spent: 0.000s
-------------------------
Na

-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 271
Time spent: 2.788s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 271
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:00:31.673276
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...
INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:02.447382
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.000688
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000784
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:02.121882
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.000514
INFO:zaml.artifa

In [28]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
#     for i in ['train', 'valid', 'test']:
    for i in ['train','test',]:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(fold_valid_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.8167179140757218}
test_auc {'auc': 0.7909094479886492}
train_ks {'ks': 0.47867797321146455}
test_ks {'ks': 0.4358379479625463}


In [33]:
ftr_impt = pd.read_parquet('/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/clientmodel1b/feature_importance.parquet')
ftr_impt.head(10)

,feature,importance
0,client_data_LTV_NORM,0.024505
1,inq_inquiries__count_by_indCode_auto_mortgage,0.007294
2,trade_months_since_openDate__mean_by_prtfType_...,0.005222
3,trade_hc_to_cl_all_non_derog_accts__max_by_acc...,0.004717
4,inq_inquiries__sum,0.004683
5,trade_months_since_openDate__mean_by_acctType_...,0.004357
6,inq_inquiries_in_last_month__sum,0.004318
7,trade_hc_to_cl_accts_opened_12m__max_by_prtfTy...,0.004274
8,trade_clAmt_active_accts__mean_by_prtfType_rev...,0.004047
9,trade_clAmt_accts_with_rcntPmt__min_by_prtfTyp...,0.003816
